Importing libraries

In [2]:
import os
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Input, ZeroPadding2D, BatchNormalization, Activation, MaxPooling2D, Flatten, Dense,Dropout
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
import imutils
import numpy as np

Model for image classification

1.A 2D convolutional layer to the model with 100 filters, a filter size of 3x3, and a rectified linear unit (ReLU) activation function. The input_shape argument specifies the shape of the input image, which is 150x150 pixels with RGB color channel.

2.A max pooling layer to the model with a pool size of 2x2. This layer reduces the spatial size of the output from the previous convolutional layer by taking the maximum value within each 2x2 window.

3.A flattening layer to the model, which converts the output from the previous layer into a 1D vector that can be fed into a fully connected layer.

4.A dropout layer to the model, which randomly drops out 50% of the connections between the previous layer and the next layer during training.

5.Dense layer-a fully connected layer to the model with 50 neurons and a ReLU activation function.

In [3]:
model =Sequential([
    Conv2D(100, (3,3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2,2),
    
    Conv2D(100, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    
    Flatten(),
    Dropout(0.5),        # prevent overfitting by reducing the model's reliance on any one feature
    Dense(50, activation='relu'),
    Dense(2, activation='softmax') # Used in multiclass classification problems to convert the output of the model into a probability distribution over the classes.
])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 100)     2800      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 100)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 100)       90100     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 100)       0         
_________________________________________________________________
flatten (Flatten)            (None, 129600)            0         
_________________________________________________________________
dropout (Dropout)            (None, 129600)            0         
_________________________________________________________________
dense (Dense)                (None, 50)                6

Compiling the model Optimizer, Loss and Accuracy Metrics

In [4]:
model.compile(optimizer='adam', 
              loss='binary_crossentropy', # minimize this difference
              metrics=['acc']) # measures the percentage of correct predictions

Setting up an image data generator for training the model to  use various image augmentation techniques, such as rotation, shifting, and flipping, which are applied to the images during training to increase the variety of data the model is to be trained on.

In [7]:
TRAINING_DIR = "F:\dataset/train"
train_datagen = ImageDataGenerator(rescale=1.0/255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

train_generator = train_datagen.flow_from_directory(TRAINING_DIR, 
                                                    batch_size=10, #number of samples in each batch
                                                    target_size=(150, 150)) #resizes the images to 150x150 pixels

Found 3268 images belonging to 2 classes.


Setting up an image data generator for validation data to evaluate the performance of the model.

In [6]:
VALIDATION_DIR = "F:\dataset/test"
validation_datagen = ImageDataGenerator(rescale=1.0/255) #image transformation,Normalizing by rescaling the pixel values to be between 0 and 1

validation_generator = validation_datagen.flow_from_directory(VALIDATION_DIR, 
                                                         batch_size=10, 
                                                         target_size=(150, 150))

Found 818 images belonging to 2 classes.


Model Fitting

In [19]:
checkpoint = ModelCheckpoint('model2-{epoch:03d}.model',monitor='val_loss',verbose=0,save_best_only=True,mode='auto')

history = model.fit_generator(train_generator,
                              epochs=10,
                              validation_data=validation_generator,
                              callbacks=[checkpoint]) # After each epoch,the ModelCheckpoint callback will save the model if its validation loss is lower than the previous best. 

Epoch 1/10
 26/327 [=>............................] - ETA: 3:17 - loss: 0.9379 - acc: 0.5077

c:\Users\USER\anaconda3\envs\cvpr_zidan\lib\site-packages\PIL\Image.py:979: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


327/327 [==============================] - 237s 724ms/step - loss: 0.5740 - acc: 0.7041 - val_loss: 0.4597 - val_acc: 0.8007
Epoch 2/10
327/327 [==============================] - 220s 674ms/step - loss: 0.4284 - acc: 0.8296 - val_loss: 0.2867 - val_acc: 0.9010
Epoch 3/10
327/327 [==============================] - 232s 709ms/step - loss: 0.4048 - acc: 0.8351 - val_loss: 0.2801 - val_acc: 0.8998
Epoch 4/10
327/327 [==============================] - 229s 699ms/step - loss: 0.3748 - acc: 0.8602 - val_loss: 0.2598 - val_acc: 0.9193
Epoch 5/10
327/327 [==============================] - 215s 658ms/step - loss: 0.3832 - acc: 0.8479 - val_loss: 0.2940 - val_acc: 0.8888
Epoch 6/10
327/327 [==============================] - 204s 625ms/step - loss: 0.3607 - acc: 0.8602 - val_loss: 0.2496 - val_acc: 0.9083
Epoch 7/10
327/327 [==============================] - 203s 622ms/step - loss: 0.3472 - acc: 0.8589 - val_loss: 0.2427 - val_acc: 0.9059
Epoch 8/10
327/327 [==============================] - 201s 

Saving the model

In [20]:
model.save('Zidan_Face_mask_detection_model_2.h5')

Taking live input through camera to detect face mask

In [11]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
#  path to the model file
model_path = "./Zidan_Face_mask_detection_model_2.h5"

# Load the model 
model = load_model(model_path)

labels_dict={0:'No Mask',1:'Mask on'}
color_dict={0:(0,0,255),1:(0,255,0)}

size = 4
webcam = cv2.VideoCapture(0) #Use camera 0

# We load the xml file,  pre-trained classifier used by OpenCV to detect faces.
classifier = cv2.CascadeClassifier('F:\opencv\opencv-master\data\haarcascades\haarcascade_frontalface_default.xml')

while True:
    (rval, im) = webcam.read()
    im=cv2.flip(im,1,1) #Flip to act as a mirror

    # Resize the image to speed up detection
    mini = cv2.resize(im, (im.shape[1] // size, im.shape[0] // size))

    # detect MultiScale / faces 
    faces = classifier.detectMultiScale(mini)

    # Draw rectangles around each face
    for f in faces:
        (x, y, w, h) = [v * size for v in f] #Scale the shapesize backup
        #Save just the rectangle faces in SubRecFaces
        face_img = im[y:y+h, x:x+w]
        resized=cv2.resize(face_img,(150,150))
        normalized=resized/255.0
        reshaped=np.reshape(normalized,(1,150,150,3))
        reshaped = np.vstack([reshaped])
        result=model.predict(reshaped)
        #print(result)
        
        label=np.argmax(result,axis=1)[0]
      
        cv2.rectangle(im,(x,y),(x+w,y+h),color_dict[label],2)
        cv2.rectangle(im,(x,y-40),(x+w,y),color_dict[label],-1)
        cv2.putText(im, labels_dict[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
        
    # Show the image
    cv2.imshow('Mask_Detection',   im)
    key = cv2.waitKey(10)
    # if q key is press then break out of the loop 
    if key == 113: #The q key
        break
# Stop video
webcam.release()

# Close all started windows
cv2.destroyAllWindows()